In [ ]:
#Class Project (Adam)
#*********************Predicting the helpfulness of Amazon movie product reviews
#*********************My developed classifier assesses a dataset of reviews
#*********************I will outline my approach to predicting the helpfulness of a review
#*********************the developed classifier will assess reviews randomly from the dataset 
#*********************and predict whether a given product review is either ”helpful” or ”unhelpful.” 
#*********************Will initially use reviewer/responder metrics along with star rankings of the reviewer 
#*********************relative to the polpulation - second phase  - will use text analysis on the actual
#*********************reviews as well.  
#*********************I was able to achieve up to ___ accuracy on movie reviews.
#   Data Source: Amazon movie reviews past 12 years

In [12]:
#preparing the data and creating a pandas data frame
import pandas as pd
with open(r'/Users/Adam/Documents/DataClass/project/movies_small.txt') as f:
    dict_list = []
    record_dict = {}
    for line in f:
#       print line
        if line.strip() is '':
            dict_list.append(record_dict)
            record_dict = {} # reset record_dict
        else:   
            split_values = line.split(': ',1)
            key = split_values[0]
            try:
                value = split_values[1].rstrip()
            except IndexError:
                value = ''
            record_dict[key] = value

df = pd.DataFrame(dict_list) # store in dataframe
df.head()

,product/productId,review/helpfulness,review/profileName,review/score,review/summary,review/text,review/time,review/userId
0,B003AI2VGA,7/7,"Brian E. Erland ""Rainbow Sphinx""",3.0,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic...",1182729600,A141HP4LYPWMSR
1,B003AI2VGA,4/4,Grady Harp,3.0,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...,1181952000,A328S9RN3U5M68
2,B003AI2VGA,8/10,"Chrissy K. McVay ""Writer""",5.0,This movie needed to be made.,The scenes in this film can be very disquietin...,1164844800,A1I7QGUDP043DG
3,B003AI2VGA,1/1,golgotha.gov,3.0,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...,1197158400,A1M5405JH9THP9
4,B003AI2VGA,1/1,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",3.0,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess...",1188345600,ATXL536YX71TR


In [13]:
#df.append(df['review/helpfulness'].apply(lambda x: pd.Series(x.split('/'), index = x.index)))
df = df.join(pd.DataFrame(df['review/helpfulness'].str.split('/').tolist(), columns=['num_helpful','num_total']).astype(int))
df.drop('review/helpfulness', inplace=True, axis=1)
#df.loc[(df['num_total'] > 200)]

In [16]:

df1 = df[['review/score','num_helpful','num_total']]
df1.head()

,review/score,num_helpful,num_total
0,3.0,7,7
1,3.0,4,4
2,5.0,8,10
3,3.0,1,1
4,3.0,1,1


In [5]:
# How many products were reviewed? How often? Distribution of products reviewed. Plot it.
import matplotlib.pyplot as plt
df.groupby('product/productId').size()

product/productId
0310582792      1
073600193X      3
0764004492      3
0767002652     24
076780046X     64
0767805534    124
0767825454      1
0767845234     10
078401115X     85
0784011532     57
0788600222     12
0790701251    315
0790732033     37
0790733382     32
0790747324    189
0790750708     68
0792151712     10
0792845773     25
0793903335      4
0793927560      1
0800103688     77
0800126033     12
0849986737      1
0934837120      2
0940871491      1
0955766818      1
0963199668      1
0966012224      1
0966199464      1
0967877709      2
             ... 
B008IG0F7G      5
B008JAG4GW     17
B008LYCDW0      1
B008M34Y72     16
B008NNY8CW      3
B008Q00I2Q    132
B008R52L48      1
B008R52L4I      1
B008ROG898      7
B008SAPAYA      6
B008SAPAYU      1
B008UO5U9E    136
B008X1O7KM    206
B008YKQC6E      2
B00925T8YI      1
B0092MDYV4      8
B00932ADHQ     11
B0094MOQ0K     14
B0095D5454    158
B0096RC57O      4
B0099VUXCQ      1
B009B6XA1U     26
B009CWWCYO      1
B009D0AZI0

In [7]:
df.groupby('product/productId').size().count()

1287

In [6]:
df.groupby(['review/userId','review/score']).size()

review/userId   review/score
A10033ZS10JQYA  4.0             1
A10078GF4Z3GJN  5.0             1
A1007T2IVFQKYN  5.0             1
A100JCBNALJFAW  3.0             1
                5.0             1
A100MPD67FD3ID  5.0             1
A100PY9HC7VVI8  5.0             1
A100RFQHFO005W  3.0             1
A100Y8WSLFJN7Q  5.0             1
A101LE0G4T2RD0  5.0             1
A101PLUE57KUSS  5.0             1
A101XQF0IGPEO   5.0             1
A1022GOAE7J1R1  5.0             1
A1023CFGMARBHM  1.0             1
A1027EV8A9PV1O  5.0             1
A102TBYVX6319C  3.0             1
A102YM3H3O003E  5.0             1
A1031BCVB4XNKB  1.0             1
A1033RWNZWEMR5  5.0             1
A103EXN5Q7HX6Z  1.0             1
A103IUREF6NQ5O  4.0             1
A103NIIU3EJZIG  5.0             1
A103QX7NUHBOUF  5.0             1
A103VZ3KDF2RT5  1.0             1
A103W7ZPKGOCC9  5.0             2
A103ZG6ASSR7UT  3.0             1
A1041HQGJDKFG5  5.0             1
A1044O5YTKM1AJ  4.0             2
A104J0XKZR32JS  5.0